In [2]:
# coding=utf-8
import sys
from os.path import abspath, join, dirname, exists
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import time
import jieba
import random
import csv
import difflib
import ast
import json
import nest_asyncio

sys.path.insert(0, join(abspath(dirname('__file__')), '../../../'))

import application as app
from chatgpt import gpt_utils
from extensions import utils

from models.analyze.analyze_tool import used_time
from models.nlp.common import text_normalize
from models.analyze.trie import Trie
from models.tiktok.video_common import *

In [3]:
dy2 = app.connect_db('dy2')

db_sop = app.get_clickhouse('chsop')
db_sop.connect()

Connect db 10.21.90.164 douyin2
Connect clickhouse 10.21.90.164 sop


In [4]:
categorynm = '运动鞋/运动穿搭'
cat = 3000003
subcid = 0

In [5]:
from paddlenlp import Taskflow

cls = Taskflow("text_classification", task_path=f'./models/{cat}/export', is_static_model=True)

[2023-12-12 16:34:36,813] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load './models/3000003/export'.
W1212 16:34:37.020805 145957 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 11.2, Runtime API Version: 11.2
W1212 16:34:37.023361 145957 gpu_resources.cc:149] device: 0, cuDNN Version: 8.1.
[2023-12-12 16:34:39,421] [    INFO] - Load id2label from ./models/3000003/export/id2label.json.


In [6]:
cls("2023,那就把3个头像捏在一起吧!#抖音年度榜单 #手工diy #泥巴玩偶 #元旦快乐 抖音年度榜单;手工diy;泥巴玩偶;元旦快乐")

[{'predictions': [{'label': '不是运动穿搭', 'score': 0.9998054860617097}],
  'text': '2023,那就把3个头像捏在一起吧!#抖音年度榜单 #手工diy #泥巴玩偶 #元旦快乐 抖音年度榜单;手工diy;泥巴玩偶;元旦快乐'}]

In [4]:
def get_data(awmids, multi_tasks = [1, 2, 3, 12, 14], prefix='', category='6'):
    multi_data = {}
    # multi_data: [Dict[aweme_id, Dict[task, List[txt, *info]]]]
    ids = ','.join(awmids)
    for task in multi_tasks:
        sql = data_sql[task].format(version=prefix, category=category, ids=ids)
        if task == 11:
            dataa = db_sop.query_all(sql, print_sql=False)
        else:
            dataa = dy2.query_all(sql)
        for aweme_id, *info in dataa:
            if multi_data.get(aweme_id) is None:
                multi_data[aweme_id] = {task: [] for task in multi_tasks}
            multi_data[aweme_id][task].append(info)

    return multi_data

def get_data_source(dy2, cat, prefix=''):
    tblnm = f"douyin_video_zl{prefix}_{cat}"
    sql = f"select data_source from {project_table} where table_name='{tblnm}';"
    rr = dy2.query_all(sql)
    return rr[0][0] if rr[0][0] else None

def get_project_cat(dy2, category, prefix):
    tbl_nm = f"douyin_video_zl{prefix}_{category}"
    sql = f'''
    select brand, prop from douyin2_cleaner.project
    where table_name='{tbl_nm}'
    '''
    rr = dy2.query_all(sql)
    if not rr:
        return category, category
    else:
        brand = rr[0][0] if rr[0][0] else category
        prop = rr[0][1] if rr[0][1] else category
        return brand, prop

In [5]:
def split_id(dy2,run_table, col, split):
    sql = f"select count(distinct {col}) from {run_table}"
    data = dy2.query_all(sql)
    count = data[0][0] if len(data) > 0 else 0
    batch_size = count // split
    current_index = 0
    durations = []
    while current_index < count:
        sql = f"select {col} from {run_table} order by {col} limit {current_index}, 1;"
        aweme_id = dy2.query_all(sql)[0][0]
        durations.append(aweme_id)
        current_index += batch_size
    durations[0] = 0
    durations[-1] = -1
    return [(durations[index], durations[index + 1]) for index in range(len(durations)-1)]

In [6]:
def count_chars(s):
    chinese_chars = 0
    english_chars = 0
    if not s:
        raise ValueError("error")

    for char in s:
        if char.isalpha():
            if char.isascii():
                english_chars += 1
            else:
                chinese_chars += 1

    return chinese_chars/len(s)

In [7]:
split = 4
seperate = 0

multi_tasks = [1, 2, 3, 14]
task_names = [type2name[task] for task in multi_tasks]

brand_cat, prop_cat = get_project_cat(dy2, cat, '')
data_type = get_data_source(dy2, cat)
print('data_type', data_type)
if data_type in (2, 3):
    multi_tasks = [1,]
    data_sql.update(xiaohongshu_data_sql)

data_type None


In [8]:
awm_tbl = f'douyin2_cleaner.awms_douyin_tmpp_{cat}'
awm_rlt_tbl = f'douyin2_cleaner.awms_douyin_result_{cat}'
rlt_cols = ['aweme_id','`type`','label','confidence']

if data_type in (2, 3):
    awm_tbl = f'douyin2_cleaner.awms_xhs_tmpp_{cat}'
    awm_rlt_tbl = f'douyin2_cleaner.awms_xhs_result_{cat}'

In [ ]:
labeltoid = {f'不是{categorynm}':0, '{categorynm}':1}

In [9]:
start,end = split_id(dy2, awm_tbl, 'aweme_id', split)[seperate]

start_condition = f"aweme_id > {start}"
end_condition = f"aweme_id <= {end}" if end > 0 else "1"

In [10]:
res = []
taskorder = [14,3,2,1]

def dy_callback(data):
    awmids = [str(x[0]) for x in data]
    if not awmids:
        return
    multi_data = get_data(awmids, multi_tasks = multi_tasks, category=cat)
    
    rawtxt = []
    for aweme_id, all_data in multi_data.items():
        txtflag = 0
        for task in taskorder:
            if task in all_data:
                data_list = all_data[task]
                tt = ','.join([text_normalize(txt) for txt, *info in data_list])
                if tt and count_chars(tt)>0.5 and len(tt)>20:
                    rawtxt.append((aweme_id, task, tt))
                    txtflag = 1
                    break
        if not txtflag:
            res.append([aweme_id, 0, -1, 0])

    data_length = len(rawtxt)
    batch_size = 20
    num_batches = -(-data_length // batch_size)

    for batch_data in tqdm(gpt_utils.load_data(rawtxt, batch_size), total=num_batches):
        txts = [x[2] for x in batch_data]
        result = cls(txts)
        for bb, rr in zip(batch_data, result):
            res.append(list(bb)[:2]+[labeltoid[rr['predictions'][0]['label']], int(rr['predictions'][0]['score']*100000000)])

sql = f'''
select aweme_id
from {awm_tbl}
where {start_condition} and {end_condition}
and aweme_id>%d
order by aweme_id
limit %d;
'''
utils.easy_traverse(dy2, sql, dy_callback, 0, 3000, print_sql=False)

100%|████████████████████████████████████████████████████████████████████| 146/146 [00:10<00:00, 13.93it/s]


start: 7183638589178121506  count: 0  total:10.90  average(1000):3.63


100%|████████████████████████████████████████████████████████████████████| 146/146 [00:13<00:00, 10.68it/s]


start: 7183970071658745088  count: 1  total:24.95  average(1000):4.16


100%|████████████████████████████████████████████████████████████████████| 146/146 [00:18<00:00,  7.85it/s]


...

100%|████████████████████████████████████████████████████████████████████| 149/149 [00:40<00:00,  3.64it/s]


start: 7216552286611426593  count: 133  total:4955.75  average(1000):12.33


100%|████████████████████████████████████████████████████████████████████| 149/149 [00:38<00:00,  3.82it/s]


start: 7216656692774178080  count: 134  total:4996.19  average(1000):12.34


100%|██████████████████████████████████████████████████████████████████████| 58/58 [00:13<00:00,  4.25it/s]


In [11]:
res[0]

[7183343151527038223, 0, -1, 0]

In [12]:
utils.easy_batch(dy2, awm_rlt_tbl, rlt_cols, res, ignore=True)

insert  ignore  into  douyin2_cleaner.awms_douyin_result_2000008  success count: 406159
